# Разметка карт

In [9]:
import folium
import pickle

In [107]:
with open('data/districts4_9.pickle', 'rb')  as f:
    places = pickle.load(f)

In [102]:
l = []
for v in places['map6'].values():
    l.extend(v)
lats = [i[0] for i in l]
longs = [i[1] for i in l] 

In [103]:
#создаем карту
m = folium.Map([37, 0], zoom_start=1, tiles='') 
folium.raster_layers.ImageOverlay(
    name='Mercator projection SW', # название слоя
    image='https://yupest.github.io/nti/site/final1/maps/map6.jpg', #ссылка на изображение карты
    bounds=[[-90, -180], [90, 180]], #обязательный параметр: наложение по координатам, границы слоя
    opacity = 1, # прозрачность карты, где 1 - непрозрачно, а 0 - прозрачно, 0.5 - половина прозрачности
).add_to(m)
folium.LatLngPopup().add_to(m)
for lat, lon in zip(lats, longs):
  
    folium.Marker(
        location=[lat, lon], # координаты широты и долготы
        icon=folium.Icon()
    ).add_to(m)
m

In [101]:
places['map6']= {
 'Набережная':[
    [-32.3290, -33.6443],
     [-18.4107, 91.5496],
     [-74.8789, -108.1980],
     [ 51.7411, 137.2665]
    ],
 'Парк':[
    [81.9430, 40.2060],
    [76.1959, -147.6778],
    [ 74.0200, 4.5772]
 ],
 'Другое':[
    [-42.9937, 156.8850],
     [-80.5053, 163.9183],
     [18.7039, 19.0310],
     [64.7822, -144.8465],
     [ -82.0871, -56.8146],
     [ 56.5758, 100.6180],
     [-82.0952, 97.8047],
     [ -45.9427,44.4144],
     [48.5119, -60.3830],
     [ -63.1775, -113.1963],
     [ -56.1248,10.9711],
     [-81.4874,-93.4398],
     [0.1315,-53.2979],
     [83.3656,94.3513],
     [ 64.8072,30.3991],
     [9.1683,-144.1432],
     [-59.7872, -72.3395],
     [ 71.0768, 148.4449],
     [78.3475, 123.1248],
     [-77.8662, -25.9193],
     [-73.3879, 53.4945],
     [ 71.9674, -65.3694]
     
 ]           
}

In [99]:
with open('data/districts4_9.pickle', 'wb') as f:
     pickle.dump(places, f)

# Генерация сайта

In [125]:
import random
import os

In [203]:
def generate_density(cat, district, problem):
    week = ['Понедельник', "Вторник", "Среда","Четверг", "Пятница", "Суббота", "Воскресенье"]
    density = '<div class="chart">\n\
        <b>Посещаемость:</b>\n'
    
    avg_density=70
    models = [True,False]
    model = models[random.randint(0,1)]
    like = random.randint(40, 100)
    
    if problem:
        problem = False
    else:
        if district == 4 and cat == 'Развлекательный центр':
            model = False
            avg_density=30
            problem=True
            like = random.randint(10, 25)

        if district == 5 and cat == 'Парк':
            avg_density=30
            problem=True
            like = random.randint(10, 25)

        if district == 6 and cat == 'Жилой квартал':
            avg_density=30
            problem=True
            like = random.randint(10, 25)
        
    for w in week[:-2]:
        if model:
            #в будни больше
            d = random.randint(int(avg_density/2),avg_density)
            d2 = random.randint(int(avg_density/4),int(avg_density/2))
        else:
            #в выходные больше
            d = random.randint(int(avg_density/4),int(avg_density/2))
            d2 = random.randint(int(avg_density/2),avg_density)
            
        density+='<div><div class="line">'+w+'</div>\n\
            <div class="pipe line">\n\
            <div style="width: '+str(d)+'%"> </div>\n\
            </div></div>\n'
        
    for w in week[-2:]:
        density+='<div><div class="line">'+w+'</div>\n\
            <div class="pipe line">\n\
            <div style="width: '+str(d2)+'%"> </div>\n\
            </div></div>\n'   
        
    return density+'</div>\n', problem, like

In [116]:
    
def get_text(list_rand):
    return list_rand[random.randint(0, len(list_rand)-1)]
    
def get_negative(category, date, time, location):
    
    dict_words=['Дизлайк!', "Не рекомендую.", "", ':(', "Ужас...", "Жалко..."]
    dict_people = ['семьей', 'друзьями', 'с подругой', "с бабушкой", "с дедушкой", " с другом", "компанией", "своей большой семьей"]
    do_trk = ['по магазинам пройтись', 'прогуляться', 'купить вещи', 'погулять', "покушать"]
    neg_m = ['не понравился', 'ужасен', "так себе", "не очень", "кошмар"]
    wish = ['все было для людей.',"сделали вход для инвалидов.", "передавалась атмосфера благоприятная.", "продавали одежду для молодежи.", "магазины не были такими старыми, мода движется вперед!"]
    
    #ТЦ
    
    temp=['ТЦ', "тц", "развлекательный центр", "ТРК"]
    
    review1 = '''%s решили выбраться в %s %s, чтобы %s, %s. В итоге %s %s! 
     Хотелось, чтобы %s''' % (
        get_text(dict_people), date, location, get_text(do_trk), get_text(do_trk), get_text(temp), get_text(neg_m), get_text(dict_words))
    
    review2  = '''Кто регулирует музыку в ТЦ %s?
    Это просто беспредел!
    Можно же музыку в фоновом режиме!
    Они же включают на максимальную громкость! %s''' %(locations, dict_words[random.randint(0, len(dict_words)-1)])
    
    
    review3 = '''Хожу в этот %s уже более 4 месяцев/
    Lа, согласен комплекс красивый и большой.
    Но количество закрытых магазинов поражает!!!!
    Отсюда вывод наверно безумная аренда!
    Жаль а могли бы быть лучшем комплексом в округе.'''%(temp[random.randint(0, len(temp)-1)])
    
    review4 = '''Охрана ТЦ %s ведет себя как сторож в колхозе. %s на первом этаже приблизительно в %s мою жену из ТЦ 
    вывел охранник с применением силы, говоря что моя супруга находится в состоянии алкогольного опъянения и на возражение "Я жду мужа из туалета" 
    он выталкал мою жену из ТЦ.''' % (location, date, time)
    
    #Парки
    
    review5 = '''Очень громко со всех сторон играла музыка, так, что хотелось бежать, и дети прикрывали уши руками. Обстановка напряженная. 
    По парку ходят торговцы цветными надувными шариками, и дети начинают ныть и просить - разводилово. Покататься и поиграть детям почти негде - 
    одна примитивная винтажная каруселька за 250 рублей с ребенка - на порядок дороже заграничных аналогов. Туалет закрыт, в рестораны очереди, 
    только банальная детская площадка, которые можно найти во многих дворах, неизменно приветливо встречает детей, но для этого не стоит тащиться в центр. 
    Много людей, суетно, противно. %s''' %(dict_words[random.randint(0, len(dict_words)-1)])
    
    review6 = '''Возможно, когда парк доделают он будет похож "на современный парки", но пока это старые развалины даже без возможности прокатиться на аттракционах. 
    Пуфики и шезлонги очень милы, но ради них ехать так далеко нет смысла.
    Лично для меня, на данный момент администрация парка убила все хоть немного хорошее, что в нем было. %s''' %(dict_words[random.randint(0, len(dict_words)-1)])
    
    review7 = '''Народу - толпы. Парковка заполнена до предела. На аттракционы (прокат веломобилей и катамаранов) огромные очереди. 
    При этом свободных веломобилей полная площадка. Видимо сотрудники не получают процент с продаж, поэтому им наплевать на клиентов. 
    В общепиты тоже кое-где очереди, но меньше. С голоду не умрете. Хотя найти мороженое вечером - проблема. Я купил последнее во всем парке! 
    Вайфай кое-где ловится, но не работает. Ну и ладно, 3G решает.
    В целом интересно посетить, погулять и посмотреть, но не ждите отдыха. Речь идет про вечер рабочего дня часов в 8-9.  %s''' %(dict_words[random.randint(0, len(dict_words)-1)])
    
    review8 = '''Прекрасный парк, на глазах меняющийся в лучшую сторону! 
    Неприятно удивили лишь некоторые цены, например в палатке с различными чаями, предложили напиток - стакан "родниковой воды" с 
    небольшим количеством ягод (морошка, брусника) на дне за 300 рублей. Космическая цена для не слишком вкусного утоления жажды. 
    Персонал приветлив, но желания возвратиться нет!%s''' %(dict_words[random.randint(0, len(dict_words)-1)])
    review13 = 'не хватает освещения %s' % (get_text(dict_words))
    #ЖК
    
    review9 = '''Очень тяжело ездить на работу.
    Если ехать на авто, то только центра будете ехать приблизительно час.
    Обратно то же самое, бывает совсем плохо.
    Если на общественном транспорте, то до пересажки минут 40-50, а потом ещё 25 минут. Народу тьма, очень тяжелая дорога. %s''' %(dict_words[random.randint(0, len(dict_words)-1)])
    
    review10 = '''Тяжёлое место. Жила там пару месяцев. Достаточно чисто, тихо, только ветер слышно. Ветренно настолько, что кажется унесёт вас на край земли.
    Дешёвые квартиры. Дешевле, чем в других местах. Многие покупают квартиру в ЖК %s и ездят по утрам на работу в соседние районы. Так сделала и я.
    Тяжело только потому, что неуютно. Сходить особо не куда. На выходных всё равно приходилось ездить в другие места развлекаться. 
    Кино нет, так же как и театра самого малюсенького. Зато школы и детские сады.''' % (location)
    
    review11 = '''Попадаешь в место домов-высоток разных цветов, расположенных вблизи друг от друга. Конечно куча магазинов, салонов красоты. Всё рядом.
    Есть скамейки, аллеи с зелеными участками, дизайн территории продуман. Но есть одно но. Урны полные мусора. Возле некоторых магазинов валяется мусор. 
    Где-то недостроены дороги, без асфальта, все в грязи. Никто не убирает улицы.  %s''' %(dict_words[random.randint(0, len(dict_words)-1)])
    
    review12 = '''Отсутствие инфраструктуры, нет работы, постоянные пробки, неблагополучная криминогенная ситуация. 
    Есть только пара более-менее просторных улиц, остальные явно узкие. %s''' %(dict_words[random.randint(0, len(dict_words)-1)])
    
    reviews = []
    
    positive = ['хорошая', "отличная", "замечательная"]
    
    malls = [review1, review2, review3, review4]
    parks = [review5, review6, review7, review8, review13]
    houses = [review9, review10, review11, review12]
    how = ['культурно', "спокойно", "познавательно"]
    
    reviews = {
        'Развлекательный центр':malls,
        'Парк':parks,
        
        'Жилой квартал':houses,
        'Достопримечательность':[
            'Все ок, но почему нельзя мыть почаще, грязно же? Неухоженный вид. Туристы приезжают а грязно: птички постарались',
            'Достопримечательность нашего района! Освещение ужасное. Вечером можно врезаться.',
            "Вандалы испортили %s. Но это было 10 лет назад. Неужели всем все равно на %s" % (get_text(dict_words), location)
        ],
        'Набережная': [
            '%s не предназначена для вечера. Освещение плохое, а тропинок просто не видно.  А так, днем освещение в порядке, гулять можно' % (location),
            '%s набережная %s, но нет пандусов для инвалидов. Сделайте доступную среду доступней!' % (get_text(positive), location),
            '%s всей семьей хотели погулять, но пошел дождь, погулять не получилось. Погода в Энске дождливая. А из-за этого размывает тропинки'  % (get_text(dict_words)),
            'Тропинки очень размывает дождями. Нет аккуратности в этой набережной. Доступную среду рисовали, но не получилсоь',
            
        ],
        'Культура':
        [
            'Обожаем %s проводить время %s. Большой ценник, у нас в районе много подростков, но они не потянут такой ценник. Да и скучно им наверно здесь' % (get_text(dict_people), get_text(how)),
            '%s Старое обветшалое здание %s' % (get_text(dict_words), get_text(dict_words)),
        ]
    }
    
    random_ID = random.randint(0, len(reviews[category])-1) 
    review = reviews[category][random_ID]
    
    if review in used_reviews[category]:
        review = get_negative(category, date, time, location)
        
    used_reviews[category].append(review)
    
    return review

In [117]:
def get_problem_review(cat, i, loc):
    reviews = {
        'Развлекательный центр':[
            'Были в ТРК %s случайно. В нем скучно. Давно хотели посетить парк аттракционов, который рядом, но пошел дождь. Пришлось зайти внутрь. ' % (loc),
            'Магазины в ТРК очень старые. Одежда совсем не молодежная. Хотели угодить всем, но получилось как всегда. Музыка неприятная играет. Скучно. ',
            'Парковка очень неудобная, дорогая. Парковка должна быть для людей, стоить не как состояние Скруджа! Всем автомобилистам сочувствую:('
        ],
        'Парк':[
            'Недавно сделали туалеты. И даже с возможностью для инвалидов. Однако, они закрыты! Ну, молодцы! А так парк больше не развивается. Много зелени. Для кого-то много зелени плюс, для меня куча зелени - негативно, ветки лезут в глаза. Нет инфраструктуры, кафе. Захочешь кофе попить кафе и гулять в парке, а не получится!',
            'Парк очень поздно открывается. Хочется бегать по утрам. А вечером у меня не получается. Зачем так придумали? Давайте бегать по утрам вместе! Давайте напишем петицию, чтобы можно было бегать по утрам.',
            "Я мама в декрете. Но по утрам не могу гулять с ребенком в парке. Он зеще закрыт, очень поздно открывается. Зелени поросло. Совсем неухоженный парк. Нет кафе, торговых точек. Хочется чтобы парк развивался, местоположение огонь! Жалко его, раньше было лучше."
        ],
        'Жилой квартал': [
            "Не живу в этом квартале. Отсутствует всяческая инфраструктура: где магазины? Элементрано в магазины ходят чтобы еды купить. Рядом скучно. Бетонные тоскливые джунгли настроили. А говорят, даже нет парковок",
            'Магазины в этом жилом квартале в дефиците видимо. Открылась 1 торговая точка. Но это же не делает жизнь легче. Он закрывается раньше, чем я дома появляюсь. Очень рано закрывается. Скучно как для детей, так и для взрослых, но последние хотябы на работе. Если хотите, чтобы молодая семья клевала на предложение, то откройте хотя бы магазин видеоигр.',
            "купили квартиру, мы молодая семья. Думали отдаленность от центра не преграда, будем ездить на машине. Но в этом квартале нет парковок! Их еще не построили. Ждем парковок!"
        ]
    }
    return reviews[cat][i]

In [118]:
def get_positive(category, date, time, location, bill):
    
    if bill =='':
        bill = 0
    dict_words=['Ставлю свой заслуженный лайк!', "Всем рекомендую!", "", 'Это нужно видеть самому)', "Приходите, не пожалеете.", "Тысячу раз рад здесь родиться"]
    
    #ТЦ
    
    review1 = '''В развлекательный центр %s бываем достаточно часто, все-таки здесь делать покупки получается реально выгоднее, чем во многих других местах. 
    Можно даже всей семьей отправляться сюда в выходной день - всем найдется занятие по вкусу. Бутики тут наверно на все случаи жизни, 
    можно купить все желаемое, и в одном месте. Так что рекомендую! %s''' % (location, dict_words[random.randint(0, len(dict_words)-1)])
    
    review2 = '''Самый любимый развлекательный центр. Из минусов – платная парковка, из-за которой сложно припарковаться стало. 
    А в остальном одни плюсы. Выполнен в европейском стиле, нет излюбленного многими предпринимателями "колхозного"оформления, когда на каждых 5 см. кто во что горазд. 
    Представлены различные кафе. Радует большое количество санитарных комнат, где всегда чисто, есть мыло, работают сушилки для рук. 
    Проводятся различные фестивали и ярмарки. для детей есть 
    очень много развлечений: качели, карусели, и поезд, который покатает ваших детей, на первом этаже есть прокат автомобилей и гироскутеров для детей, 
    так же есть огромный супермаркет Лента, где цены самые низкие. %s''' %(dict_words[random.randint(0, len(dict_words)-1)])
    
    review3 = '''Всем доброго дня! Я уже не первый раз посещаю развлекательный центр %s. В принципе меня там всё устраивает. 
    Удобно добираться, чисто, свободно, большой ассортимент на любой вкус. Но при последнем посещении я обнаружила, что на 10- той линии открылся новый 
    павильон - домашний текстиль от Ивановской фабрики. Халатики, пижамы, туники и др. большлй ассортимент, а цены как удивили - низкие! 
    Можно купить и оптом и в розницу. %s''' % (location, dict_words[random.randint(0, len(dict_words)-1)])
    
    review4 = '''Мы тоже очень полюбили этот центр %s, хотя поначалу переживали по поводу прежнего кинотеатра. 
    Мало похожих развлекательный центр с такой удобной планировкой. Тут всегда просторно, в любое время дня. Как-то продуманы потоки посетителей, что всем хватает места. 
    А уж 4-ый этаж просто красота. Балкон со столиками очень радует. %s''' % (location, dict_words[random.randint(0, len(dict_words)-1)])
    
    #Парки
    
    review5 = '''Хороший стал парк, есть где погулять с детьми, покушать, поиграть, посмотреть, всё красиво и ухожено! %s''' %(dict_words[random.randint(0, len(dict_words)-1)])
    
    review6 = '''Очень спокойное, интересное место, где можно отдохнуть , заняться спортом, провести выходные и великолепные ночи в открытом кинотеатре. %s''' %(dict_words[random.randint(0, len(dict_words)-1)])
    
    review7 = '''Одно из лучших мест в нашем городе после реконструкции. Просто пройтись после работы, посидеть на набережной, выпить кофе и сбросить напряжение за день. 
    Всё продумано и сделано для людей, на любой вкус — от теннисных столов и танцевальной эстрады до фотографии и кино. Чисто, уютно и улыбчиво:) Спасибо! %s''' %(dict_words[random.randint(0, len(dict_words)-1)])
    
    review8 = '''Парк, конечно, замечательный. Но сейчас мне больше всего в нем нравятся танцевальные площадки, которые находятся на набережной у пешеходного моста через 
    реку. Все семь дней в неделю можно придти и потанцевать в паре, если Вы умеете, или придти чуть пораньше и поучиться. Какая же это романтика, 
    когда летом ты танцуешь под вечереющим небом, а в реке отражаются огни подсветки и фонарей... и плывут кораблики :) %s''' %(dict_words[random.randint(0, len(dict_words)-1)])
    
    #ЖК
    
    review9 = '''Жилой комплекс «%s» достаточно новый, примерно 2016 года постройки. Состоит из одного центрального 7-ми этажного здания, нескольких таунхаусов и 
    нескольких квадрохаусов. 
    То есть это очень удобная малоэтажная застройка из домиков в три этажа. Домики на достаточно обширной территории, которая огорожена забором, под охраной и видеонаблюдением. 
    Своя детская площадка, спортивный корт. Много мест для парковок, зеленые газоны и клумбы.''' % (location)
    
    review10 = '''Преддомовая территория покрыта качественным газоном причем настоящим, детская площадка просто загляденье, огромная парковка за жилой зоной, 
    пешеходные дорожки выложены 
    тратуарной плиткой, уличное освещение просто замечательное, всё хорошо. %s''' %(dict_words[random.randint(0, len(dict_words)-1)])
    
    review11 = '''Мы живем тут уже три года. Транспортная доступность здесь отличная. На 19 автобусе, который останавливается возле дома, 
    мы можем доехать практически до любого социального объекта в городе, будь то больница, МФЦ, налоговая и многое другое.  
    Детская и взрослая поликлиника находится в одном из домов ЖК спутник, есть детский сад, множество секций и кружков для детей и взрослых, 
    очень не плохой государственный лицей, сетевые магазины и многое другое.'''
    
    review12 = '''Купили квартиру здесь 2,5 года назад. район очень нравится. В первую очередь подкупает простор и тишина. Живем сколько времени, и наслаждаемся этим 
    все больше и больше. Даже карантин, когда все дома сидели, у нас как то легко прошел, мы гуляли, здесь людей не много, в подъезде опять же чисто. Кстати, управляющая 
    компания здесь работает отлично, возле домов чистота, в подъездах каждый день убирают. %s''' %(dict_words[random.randint(0, len(dict_words)-1)])
    
    #Набережные
    
    review13 = '''Можно спуститься к морю. Пляж галечный. Водичка прозрачная. Можно просто постоять у парапета, вглядываясь в морскую даль.
    Попадаются деревья с пышной кроной и усыпанные цветами кусты.
    Если вы проголодались, вас ждут многочисленные заведения общепита.
    Кроме этого, есть несколько точек с развлечениями типа стрельбы по банкам из ружья и метания дротиков в воздушные шарики. %s''' %(dict_words[random.randint(0, len(dict_words)-1)])
    
    review14 = '''Очень чисто и красиво. Хорошо оформлены аллеи, клумбы. Есть чистые бесплатные туалеты на протяжении всей набережной, 
    в туалете есть вода и туалетная бумага, здесь по времени работает уборщица, что приятно удивило. Много фонарей, вечером тут красотища.
    Хорошее зонирование. Много площадок для детей. Сделано так, что можно детям бегать и падать и не беспокоиться о разбитых коленях, 
    поскольку мягкий пол. Много лазилок, качелек, лавочек. Для мамочек с детьми здесь просто идеальное место! %s''' %(dict_words[random.randint(0, len(dict_words)-1)])
    
    review15 = '''Сейчас набережная %s это место отдыха как молодежи, так и пожилых людей, а так же семей с детьми разного возраста. 
    Летом вечерами на набережной проводятся различные мероприятия, открыты разные мастер-классы и аттракционы.''' % (location)
    
    review16 = '''Можно не только погулять, но и подкрепиться. Очень понравилась большая детская площадка, где для детей установлены различные зоны для игр.
    Также приятно удивило наличие площадки с тренажёрами для разной возрастной категории.'''
    
    #Кафе
    food = ['Цезарь', "Картошка фри", "Греческий", "Креветки в панировке", "Пиццу"]
    review17 = '''Мне здесь понравилась кухня. Я заказывала блюдо «%s» осталась очень довольна качеством. Интерьер полностью соответствует названию ресторана. 
    Очень хорошее обслуживание официантов. Как ни странно в ресторане %s очень хороший и чистый туалет. Сюда пришли, так как меня больше интересовала музыка, а в остальном 
    было приятным дополнением.'''% (food[random.randint(0, len(food)-1)], location)
    
    review18 = '''Заведение замечательное, хожу сюда постоянно уже на протяжении 8 лет. В заведении вкусные блюда и напитки. Обслуживание хорошее, сотрудники вежливые и 
    работают быстро. В заведении интерьер и музыка в тематике рок энд ролла. Особенно мне нравится музыка и атмосфера. Внутри все комфортно и чисто.'''
    
    review19 = '''Периодически заглядываю на бизнес ланч,когда хочется разнообразия — очень достойное место и чек меня устраивает: %s! Приятный интерьер,все вкусно,обязательно когда нибудь устрою
    себе и своим близким гастрономический вечер и попробую все блюда основного меню)))'''% (str(bill))
    
    review20 = '''Лучшее место, где можно потусоваться в любой день недели. Цены вполне доступные, разнообразное барное меню, дымные кальяны. %s''' %(dict_words[random.randint(0, len(dict_words)-1)])
    
    #Культура
    
    review21 = '''Были здесь %s. Место интересное, цены вполне уместные. Рекомендую сходить в %s как взрослым, так и детям.''' % (date, location)
    
    review22 = '''Были там с друзьями. Всё очень понравилось, интересно провели время. Всем рекомендую.'''
    
    review23 = '''Ходили туда семьёй %s. Детям понравилось, нам тоже. Помимо самого развлечения есть ещё хорошее кафе. Не жалеем о том, что мы туда сходили.''' % (location)
    
    #Достопремечательности
    
    review24 = '''Место крайне интересное, дизайнер хорошо постарался. Молодёжь любит ходить сюда. Мне всё понравилось! %s Супер! %s''' %(location, dict_words[random.randint(0, len(dict_words)-1)])
    
    review25 = '''Гулял тут с девушкой, хорошо провели вечер. Вечером здесь включают подсветку, всё становится ещё лучше. Так что предлагаю вам посетить %s.''' % (location)
    
    review26 = '''Ходила сюда с ребёнком, хорошо погуляли. Доченьке понравилось. Думаем вернуться сюда на следующей неделе. %s''' %(dict_words[random.randint(0, len(dict_words)-1)])
    
    reviews = []
    
    malls = [review1, review2, review3, review4]
    parks = [review5, review6, review7, review8]
    houses = [review9, review10, review11, review12]
    embankment = [review13, review14, review15, review16]
    caffees = [review17, review18, review19, review20]
    culture = [review21, review22, review23]
    sights = [review24, review25, review26]
     
    if category == 'Развлекательный центр':
        reviews = malls
    
    if category == 'Парк':
        reviews = parks
    
    if category == 'Жилой квартал':
        reviews = houses
        
    if category == 'Набережная':
        reviews = embankment
        
    if category == 'Кафе':
        reviews = caffees
        
    if category == 'Культура':
        reviews = culture
        
    if category == 'Достопримечательность':
        reviews = sights
        
    random_ID = random.randint(0, len(reviews)-1) 
    review = reviews[random_ID]
    if review in used_reviews[category]:
        random_ID=random.randint(0, len(reviews)-1)
#         print(random_ID)
        review = get_positive(category, date, time, location, bill)
    used_reviews[category].append(review)
    
    return review

In [208]:
def generate_reviews(like, category, location, bill, problem):
    reviews = '<div class="comments">\n\
    <b>Комментарии:</b>\n\
    <ul class="media-list">\n'
    
    if like>60:
        start = 4
    else:
        start = 1
    
    for i in range(0,3):
        
        rating = random.randint(start,5)
        month = ['января', "февраля", "марта", "апреля", "мая", "июня", "июля", "августа", "сентября", "октября", "ноября", "декабря"]
        dateWOTime = str(random.randint(1,28))+' '+month[random.randint(0,11)]+' '+str(random.randint(2017, 2020))
        time = str(random.randint(10,23))+':'+str(random.randint(10,59))
        date = dateWOTime + ', ' + time

        if rating<4 and category in ['Жилой квартал','Парк', "Развлекательный центр"]:
            review = get_negative(category, dateWOTime, time, location)
        else:
            review = get_positive(category, dateWOTime, time, location, bill)
        
        if problem and category in ['Развлекательный центр','Парк','Жилой квартал']:
            review = get_problem_review(category, i , location)
            rating = random.randint(1,3)
            
        reviews+='<li class="media">\n\
            <div class="media-body">\n\
                <div class="media-heading">\n\
                    <div class="author"><span class="rating">'+str(rating)+'</span><span>⭐</span><span class="devide">|</span></div>\n\
                    <div class="metadata">\n\
                        <span class="date">'+date+'</span>\n\
                    </div>\n\
                </div>\n\
                <p class="review">'+review+'</p>\n\
            </div>\n\
        </li>\n'
    return reviews+'</ul></div>\n'

In [151]:
with open('data/districts4_9.pickle', 'rb')  as f:
    places = pickle.load(f)

Если попадает в бесконечный цикл - прерывать - запускать заново, да, костыльно, но поправлять некогда(

In [210]:
use_sight = []
categories = ['Парк', 'Набережная', 'Кафе', 'Развлекательный центр', 'Достопримечательность','Культура','Жилой квартал']
locations = {'Парк':['Динамо', "Комсомольский", "ЦПКиО", "им. Горького", "Дендрарий", "Победы", "Византийский сад", "Райский сад", "Вишневый сад"], 
             'Набережная': ["Масалитинова", "Макарова", "Нижняя", "Верхняя", "Постышева", "Морская", "Черепаховая", "Куликовская"], 
             'Кафе':['Рататуй', "Имбирь", "Ичираку рамен", "Виктория", "Вкус Детства", "Имбирь", "Жадина Говядина", "Байкал", "Сибирь", "Токио","Крастикраб", "Маленькая Италия", "Океан"], 
             'Развлекательный центр':["Карусель", "Боулинг", "Микс", "Мир", "Твикс", "Остров пиратов", "Карибы", "Молл"],
             'Достопримечательность':['Памятник Мстителям', "Арка искусств", "Мемориал пожарным", "Скульптура фонарщика", "Памятник великой черепахе", "Стена хокаге", "Дворец принцессы", 'Домики Шира', "Граффити СоюзМультфильм", "Кошкин дом", "Тролли"],
             'Культура':["Музей сладостей", "Театр аниме", "Кинотеатр Максима", "Драмтеатр", "КинотеатрМолл", "Музей черепах", "Музей истории", "Галлерея кинематографии", "Галерея Детроит", "Музей чипсов", "Музей Мстителей", "Театр Картинки кино", "Галерея"],
             'Жилой квартал':['Жемчужина', 'Стрижи', "Ласточка", "Зверополис", "Домино", "Изумрудный город", "Тридевятое Королевство","Гравити Фолз", "Готэм-Сити", "Спрингфилд",]
            }
close='</div>'

for s in range(4,7): 
    html = '<!DOCTYPE html>\n\
    <html>\n\
    <head>\n\
    <title>Дистрикт Энска</title>\n\
    <meta charset="utf-8"/>\n\
    <meta name="description" content="">\n\
    <meta name="viewport" content="width=device-width">\n\
    <link href="../css/bootstrap.css" rel="stylesheet"/>\n\
    <link href="../css/templatemo-misc.css" rel="stylesheet"/>\n\
    <link href="../css/templatemo-style.css" rel="stylesheet"/>\n\
    <link href="http://fonts.googleapis.com/css?family=Roboto+Slab:400,700,300,100" rel="stylesheet"/>\n\
    <link href="http://fonts.googleapis.com/css?family=Roboto:400,400italic,300italic,300,500,500italic,700,900" rel="stylesheet"/>\n\
    <link rel="stylesheet" href="../css/font-awesome.css">\n\
    <link rel="stylesheet" href="../css/animate.css">\n\
    <script src="../js/vendor/modernizr-2.6.1-respond-1.1.0.min.js"></script>\n\
    </head>\n\
    <body>\n\
    <div class="content-wrapper">\n\
    <div class="inner-container container">\n\
        <div class="row">\n\
            <div class="section-header col-md-12">\n\
                <h2>Дистрикт Энска</h2>\n\
                <span>район города Энск</span>\n\
            </div>\n\
        </div>\n\
        <div class="project-detail row">\n\
            <div class="project-slider col-md-12">\n\
                <img src="greens/green'+str(s)+'.jpg" alt="Фото с квадрокоптера">\n\
                <img src="../../Индексы качества городской среды.png">\n\
                <img src="maps/map'+str(s)+'.jpg" alt="Карта района">\n\
                <iframe src="map_district'+str(s)+'.html" width="100%" height = "100%" scrolling="auto"> </iframe>\n\
                <a href="#" class="slidesjs-previous slidesjs-navigation">&lt;</a> \n\
                <a href="#" class="slidesjs-next slidesjs-navigation">&gt;</a>\n\
            </div>\n\
            <div class="project-infos col-md-12">\n\
                <div class="box-content">\n\
                    <h2 class="project-title">Обращение мэра Энска</h2>\n\
                    <p> Энск большой развивающийся город, численность населения которого составляет 105 тыс. человек. В Энске условно комфортный климат. Недавно город принял участие в программе «Индексы качества городской среды». \
    Программа «Индексы качества городской среды» - это инструмент, который направлен  на выявление пространств, нуждающихся в изменениях в первую очередь. Программа оценивает, насколько город отвечает потребностям горожан.\
    В прошлом 2019 году Энск набрал 152 балла из возможных 360.  На рисунке 2 в слайдере вы можете видеть основные показатели городской среды Энска.\
    Я, как мэр города, очень расстроен полученными результатами. В последние три года администрация города предприняла ряд шагов к улучшению социально-досуговой инфраструктуры и прилегающих пространств.  В одном из районов (District 1) города построен современный развлекательный центр «Остров пиратов» с большим свободным пространством перед зданием, на котором располагается парк аттракционов. Собранная администрацией статистика показывает, что этот центр не пользуется популярностью, хотя нас уверяли, что запросы на подобные объекты городской инфраструктуры в городе очень высокие. В чем причина маленькой посещаемости «Острова пиратов»? Что можно предпринять, чтобы повысить эту посещаемость? Какие решения могут способствовать повышению комфорта  этого объекта? Может быть? есть примеры решения такой проблемы в других городах?\
    Город Энск стремится к процветанию и старается предложить своим жителям и гостям лучшие решения для благоприятного пребывания в нем! В 2019 году принято решение реконструкции парковых зон и мест отдыха. Ведь они являются сердцем города. Парковые зоны способствуют улучшению качества воздуха и являются средой обитания и развития представителей флоры и фауны. Также они способствуют сплочению городского населения и повышению качества его жизни. В Энске замечательный «Центральный парк культуры и отдыха». ЦПКиО был построен еще в начале прошлого века и был неизменным семейным местом отдыха горожан. Место отдыха расположено на территории прекрасного природного массива. Почему же сегодня у парка такая низкая посещаемость, а горожане жалуются на нехватку мест для семейного отдыха в этом районе (District 2). В чем причины? Чего не хватает парку? Почему он не может по прежнему собирать семью для отдыха под зеленью своих аллей?</p><p>\
    Современный город - это экосистема. Мэр и администрация Энска стараются создавать в нем наиболее благоприятные условия для жизни. В районе District 3, построен новый жилой квартал «Готем Сити», позволяющий улучшить жителям города свои жилищные условия. В строительство этого района вложено много средств. И что же в итоге? Квартиры в этом ЖК не распроданы уже в течение трех лет. Даже при наличии льготных систем кредитования покупки жилья, специальных условий приобретения жилья молодыми семьями,  горожане не спешат менять старую обветшалую жилплощадь на новую. В чем причина? Чего не хватает новому жилому кварталу? Какие есть проблемы? Можно ли предложить для администрации города решения, которые позволят улучшить мнение горожан о новом жилом комплексе? Есть ли примеры выхода из такой критической ситуации в других городах аналогичных Энску?\
    </p><p>\
     Уважаемые молодые аналитики, начинающие специалисты в сфере среды обитания, от лица мэра города Энск и всей администрации прошу Вашей помощи в решении проблем нашего города. Жду перечисления болевых точек районов, обоснованных рекомендаций и конструктивных решений.\
    .</p>\n\
                    <ul class="project-meta">\n\
                        <li><i class="fa fa-folder-open"></i>Мэр Энска</li>\n\
                        <li><i class="fa fa-calendar-o"></i>15 Ноября 2020</li>\n\
                    </ul>'+close*5+"<script src='../js/vendor/jquery-1.11.0.min.js'></script>\n\
        <script>window.jQuery || document.write('<script src='../js/vendor/jquery-1.11.0.min.js'><\/script>')</script>\n\
        <script src='../js/plugins.js'></script>\n\
        <script src='../js/main.js'></script>\n"

   
    problem = []
    use_coords = []  
    use_locs = []
    use_photos = []
    use_cats = {
        'Парк':0, 
        'Набережная':0, 
        'Кафе':0, 
        'Развлекательный центр':0, 
        'Достопримечательность':0,
        'Культура':0,
        'Жилой квартал':0
    }
    used_reviews = {'Парк': [], 'Набережная': [], 'Кафе': [], 'Развлекательный центр': [], 'Достопримечательность': [], 'Культура': [], 'Жилой квартал': []}
    
    for i in range(1, 16):
        
        cat = categories[random.randint(0,len(categories)-1)] 
        
        if sum(use_cats.values()) == 14 and s==5 and not True in problem:
            cat = 'Парк'
        elif sum(use_cats.values()) == 14 and s==4 and not True in problem:
            cat = 'Развлекательный центр'
        elif sum(use_cats.values()) == 14 and s==6 and not True in problem:
            cat = 'Жилой квартал'
        else:
            cat_temp = cat

            if cat!= 'Набережная' and cat!= 'Парк':
                cat_temp= 'Другое'

            while len(places['map'+str(s)][cat_temp])== use_cats[cat]:
                cat = categories[random.randint(0,len(categories)-1)] 
            use_cats[cat]+=1    
        
        loc = []
        if cat == 'Достопримечательность' or cat== 'Культура' or cat== 'Жилой квартал':
            while loc in use_sight or loc ==[]:
                loc = locations[cat][random.randint(0, len(locations[cat])-1)]
            use_sight.append(loc)
        else:
            
            while loc in use_locs or loc ==[]:
                
                loc = locations[cat][random.randint(0, len(locations[cat])-1)]
            use_locs.append(loc)
            
        density, p, like =generate_density(cat, s, True in problem) 
        problem.append(p)
        
        coordinates = []
        while coordinates in use_coords or coordinates == []:
            if cat== 'Парк' or cat== 'Набережная':
                coordinates = places['map'+str(s)][cat][random.randint(0,len(places['map'+str(s)][cat])-1)]
            else:
                coordinates = places['map'+str(s)]['Другое'][random.randint(0,len(places['map'+str(s)]['Другое'])-1)]
            
        use_coords.append(coordinates)

        photo = []
        while photo in use_photos or photo == []:
            if os.path.exists('site/images/'+cat+'/'+loc+'.jpg'):
                photo = loc
            else:
                files = []
                for f in os.listdir('site/images/'+cat+'/'):
                    if cat in f:
                        files.append(f)
                photo = cat+str(random.randint(1,len(files)))
        use_photos.append(photo)
        
        bill=random.randint(300, 2500)
        
        html+='<div class="col-md-4 project-item mix">\n\
            <div class="project-thumb">\n\
            <img name="category" alt="'+cat+'" src="../images/'+cat+'/'+photo+'.jpg"/>\n\
            </div>\
            <div class="box-content project-detail">\n\
            <h2 name="location">'+loc+'</h2>\n\
            <b>Координаты: </b><div class="coordinates"><span name="lattitude">'+str(coordinates[0])+'</span><span>, </span><span name="longitude">'+str(coordinates[1])+'</span></div>\n\
            <div><span name="count_likes">'+str(like)+'</span><span>❤️</span>'
        if cat == 'Кафе':
            html+='<span class="devide">|</span><span name="bill">'+str(bill)+'</span><span>₽</span>'
            
        html+='</div>\n'
        html+=generate_reviews(like, cat, loc, bill, p)
        html+=density
        html+=close*3

    html+='</body>\n\
    </html>'

    f = open('site/final1/district'+str(s)+'.html', 'w', encoding='utf-8')
    f.write(html)

# Проверека - Решение задач

Проверяем:
- посещаемость
- распределение мест на карте по лайкам и категориям
- отзывы

Парсинг

In [221]:
from bs4 import BeautifulSoup # библиотека для парсинга - поиск информации по структуре тегов
import requests
import folium

In [229]:
dfs = []
for p in range(4,7):
    f = open('site/final1/district'+str(p)+'.html', 'r', encoding='utf-8')
    html = f.read()
    # html = requests.get('https://yupest.github.io/nti/site/final1/district6.html').text
    soup = BeautifulSoup(html, 'lxml')

    locations = [r.text for r in soup.find_all('h2', {'name':'location'})]
    categories = [r['alt'] for r in soup.find_all('img', {'name':'category'})]
    text = [r.text for r in soup.find_all('p', {'class':'review'})]
    rating = [int(r.text) for r in soup.find_all('span', {'class':'rating'})]
    likes = [int(r.text) for r in soup.find_all('span', {'name':'count_likes'})]
    lattitudes = [float(r.text) for r in soup.find_all('span', {'name':'lattitude'})]
    longitudes = [float(r.text) for r in soup.find_all('span', {'name':'longitude'})]
    density = [int(el.div['style'][7:-1]) for el in soup.find_all('div',{'class':'pipe line'})]
    density_list = []
    for i in range(0, len(density), 7):
        density_list.append(int(sum(density[i:i+7])/7))
    #для участников
    den = pd.DataFrame({'location':locations, 'category':categories, 'like':likes})
    den = den.sort_values('like')
    den.to_csv('district'+str(p)+'places.csv', index = False)
    
    #для нас
    den = pd.DataFrame({'location':locations, 'category':categories, 'like':likes, 'density':density_list})
    den = den.sort_values('density')
    dfs.append(den)
    
    m = folium.Map([37, 0], zoom_start=2, tiles='')
    img = folium.raster_layers.ImageOverlay(
        name='Mercator projection SW', # название слоя
        image='https://yupest.github.io/nti/site/final1/maps/map'+str(p)+'.jpg', #ссылка на изображение карты
        bounds=[[-90, -180], [90, 180]], #обязательный параметр: наложение по координатам, границы слоя
        opacity = 0.6, # прозрачность карты, где 1 - непрозрачно, а 0 - прозрачно, 0.5 - половина прозрачности
    )
    # добавить изображение в карту
    img.add_to(m)

    for lat, lon, name, like, cat in zip(lattitudes, longitudes, locations, likes, categories):

        folium.CircleMarker(
            location=[lat, lon], 
            radius = 7, 
            popup=name.encode('raw_unicode_escape').decode('utf-8'), 
            fill_color='red', 
            color = None,
            fill_opacity = 1
        ).add_to(m)
    m.save('site/final1/map_district'+str(p)+'.jpg')
    m.save('site/final1/map_district'+str(p)+'.html')

Карты

In [240]:
dfs[2]

,location,category,like,density
5,Тридевятое Королевство,Жилой квартал,13,12
13,Театр аниме,Культура,98,31
1,Сибирь,Кафе,77,33
14,Черепаховая,Набережная,93,33
7,Виктория,Кафе,89,35
0,Ичираку рамен,Кафе,83,38
12,Масалитинова,Набережная,43,38
8,Домики Шира,Достопримечательность,59,39
11,Океан,Кафе,51,39
2,Театр Картинки кино,Культура,67,43


Проблемные места:
- Д4 - Боулинг 14% посещений
- Д5 - Победы 18 %
- Д6 - Тридевятое царство 12%


Ответ на 17 задачу
- Д4 - парк византийский сад
- Д5 - культура кинотеатрмолл
- Д6 - культура театра аниме

Ответы на 7 задачу:

- Д4 - Победы
- Д5 - Комсомольский
- Д6 - Арка икусств

Отзывы

In [ ]:
negative = []
positive = []
for i in range(0, len(rating)):
    if rating[i]<4:
        negative.append(text[i])
    else:
        positive.append(text[i])
        
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

#Выделяем слова которые точно не должны использоваться при построении облака слов!
stopwords = set(STOPWORDS)
stopwords.update(["на", "все", "эта", "не", "уже", "это", "что","много","чисто", "рекомендую", "хорошо", "можно", "понравилось","просто", "по","всем","есть", "тоже", "так", "но", 'какой', "очень", 'n', 'здесь', "место"])

In [ ]:
#Создаем облако
wordcloud = WordCloud(stopwords = stopwords, background_color="white", min_word_length=4 ).generate(str(text))

# Рисуем результат
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Все')
plt.axis("off")
plt.show()

In [ ]:
#Создаем облако
wordcloud = WordCloud(stopwords = stopwords, background_color="white",min_word_length=4 ).generate(str(negative))

# Рисуем результат
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Негативные')
plt.axis("off")
plt.show()

In [ ]:
#Создаем облако
wordcloud = WordCloud(stopwords = stopwords, background_color="white",min_word_length=4 ).generate(str(positive))

# Рисуем результат
plt.imshow(wordcloud)
plt.title('Негативные')
plt.axis("off")
plt.show()

In [ ]:
{k: v for k, v in sorted(WordCloud(stopwords = stopwords, background_color="white",min_word_length=4 ).process_text(str(positive)).items(), key=lambda item: item[1], reverse=True)}